In [16]:
!unrar x "/content/2E-EVRP-Instances-v2.rar" "/content/2E-EVRP-Instances-v2/"





UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/2E-EVRP-Instances-v2.rar

Creating    /content/2E-EVRP-Instances-v2                             OK
Creating    /content/2E-EVRP-Instances-v2/2E-EVRP-Instances-v2        OK
Creating    /content/2E-EVRP-Instances-v2/2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0  OK
Extracting  /content/2E-EVRP-Instances-v2/2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/README.md       0%  OK 
Creating    /content/2E-EVRP-Instances-v2/2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/Type_x  OK
Creating    /content/2E-EVRP-Instances-v2/2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/Type_x/Customer_10  OK
Extracting  /content/2E-EVRP-Instances-v2/2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/Type_x/Customer_10/C101_C10x.txt       0%  OK 
Extracting  /content/2E-EVRP-Instances-v2/2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/

In [17]:
import os

for root, dirs, files in os.walk("/content/2E-EVRP-Instances-v2"):
    print(root, dirs, files)
    break


/content/2E-EVRP-Instances-v2 ['2E-EVRP-Instances-v2'] []


In [18]:
!apt-get install unrar
!unrar x 2E-EVRP-Instances-v2.rar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from 2E-EVRP-Instances-v2.rar

Creating    2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0  OK
Extracting  2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/README.md       0%  OK 
Creating    2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/Type_x  OK
Creating    2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/Type_x/Customer_10  OK
Extracting  2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/Type_x/Customer_10/C101_C10x.txt       0%  OK 
Extracting  2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/Type_x/Customer_10/C102_C10x_n.txt       0%  OK 
Extracting  2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instan

In [21]:
import os
import random
import math

# ============================================================
#  LECTURE D'INSTANCE : version compatible Manilakbay v2
# ============================================================

def read_instance(filepath):
    nodes = []
    params = {}

    with open(filepath, 'r') as f:
        lines = f.readlines()

    # Trouver l'en-tête "StringID"
    header = None
    for i, line in enumerate(lines):
        if line.strip().startswith("StringID"):
            header = i
            break
    if header is None:
        raise ValueError("En-tête non trouvée dans le fichier.")

    # Lire les noeuds + paramètres
    for line in lines[header+1:]:
        parts = line.strip().split()
        if not parts:
            continue

        # ---------------------------
        #  NOUVEAU : extraction paramètres du type :
        #  Q info /60.63/
        # ---------------------------
        if parts[0] in ['L', 'C', 'Q', 'r', 'g', 'v'] and '/' in line:
            try:
                value = float(line.split('/')[1])  # valeur entre / ... /
                params[parts[0]] = value
            except:
                pass
            continue

        # ---------------------------
        #  Lecture des nœuds
        # ---------------------------
        if len(parts) >= 5 and parts[1] in ('d', 'c', 's', 'f'):
            demand = float(parts[5]) if len(parts) > 5 else float(parts[4])

            node = {
                'id': parts[0],
                'type': parts[1],
                'x': float(parts[2]),
                'y': float(parts[3]),
                'demand': demand
            }
            nodes.append(node)

    # Tri des noeuds : dépôt → satellites/chargeurs → clients
    nodes.sort(key=lambda n: 0 if n['type'] == 'd' else 1 if n['type'] in ('s','f') else 2)

    # Vérification des paramètres
    if 'Q' not in params or 'r' not in params or 'C' not in params:
        raise ValueError(f"Paramètres manquants : {params}")

    Q = params['Q']
    h = params['r']
    vehicle_capacity = params['C']

    return nodes, Q, h, vehicle_capacity


# ============================================================
#  OUTILS CALCULS
# ============================================================

def distance(a, b):
    return math.hypot(a['x'] - b['x'], a['y'] - b['y'])

def is_capacity_feasible(route, nodes, cap):
    return sum(nodes[i]['demand'] for i in route) <= cap

def compute_route_cost_and_feasible(route, nodes, Q, h):
    if not route:
        return 0, True

    total_dist = 0
    battery = Q
    prev = nodes[0]  # dépôt

    for idx in route:
        curr = nodes[idx]
        d = distance(prev, curr)
        cons = h * d

        if battery < cons:
            battery = Q  # recharge automatique
        battery -= cons
        total_dist += d
        prev = curr

    # Retour au dépôt
    d = distance(prev, nodes[0])
    cons = h * d
    if battery < cons:
        battery = Q
    battery -= cons
    total_dist += d

    return total_dist, battery >= 0


# ============================================================
#  OBJECTIF ET ALGORITHME SA
# ============================================================

def objective(solution, nodes, Q, h, cap):
    total = 0
    for route in solution:
        if not is_capacity_feasible(route, nodes, cap):
            return float('inf')
        c, feas = compute_route_cost_and_feasible(route, nodes, Q, h)
        if not feas:
            return float('inf')
        total += c
    return total

def generate_initial_solution(nodes, vehicle_capacity):
    customers = [i for i in range(1, len(nodes))]
    random.shuffle(customers)

    solution = []
    route = []
    load = 0

    for idx in customers:
        d = nodes[idx]['demand']
        if load + d > vehicle_capacity:
            solution.append(route)
            route = []
            load = 0
        route.append(idx)
        load += d

    if route:
        solution.append(route)

    return solution

def get_neighbor(solution):
    new_sol = [r[:] for r in solution]

    if len(new_sol) < 2:
        return new_sol

    r1, r2 = random.sample(range(len(new_sol)), 2)
    if not new_sol[r1] or not new_sol[r2]:
        return new_sol

    i1 = random.randrange(len(new_sol[r1]))
    i2 = random.randrange(len(new_sol[r2]))

    new_sol[r1][i1], new_sol[r2][i2] = new_sol[r2][i2], new_sol[r1][i1]
    return new_sol

def SA_algorithm(nodes, Q, h, cap, initial_temp=1000, cooling_rate=0.995, max_iter=200):
    current = generate_initial_solution(nodes, cap)
    current_cost = objective(current, nodes, Q, h, cap)

    best = [r[:] for r in current]
    best_cost = current_cost

    temp = initial_temp

    for _ in range(max_iter):
        neighbor = get_neighbor(current)
        neigh_cost = objective(neighbor, nodes, Q, h, cap)
        delta = neigh_cost - current_cost

        if delta < 0 or random.random() < math.exp(-delta / temp):
            current = [r[:] for r in neighbor]
            current_cost = neigh_cost
            if current_cost < best_cost:
                best = [r[:] for r in current]
                best_cost = current_cost

        temp *= cooling_rate

    return best, best_cost


# ============================================================
#  AFFICHAGE
# ============================================================

def afficher_solution(solution, nodes, Q, h):
    print(f"  Paramètres : Q={Q}, h={h}")
    for i, route in enumerate(solution):
        seq = " -> ".join(nodes[idx]['id'] for idx in route)
        print(f"  Tournée {i+1}: Dépôt -> {seq} -> Dépôt")
    print(f"  Nombre total de tournées : {len(solution)}")


# ============================================================
#  TRAITEMENT DES INSTANCES
# ============================================================

def traiter_toutes_les_instances(dataset_folder):
    for filename in os.listdir(dataset_folder):
        if filename.endswith(".txt"):
            fp = os.path.join(dataset_folder, filename)
            print("="*60)
            print("Traitement de", filename)

            nodes, Q, h, cap = read_instance(fp)

            n_clients = sum(1 for n in nodes if n['type'] == 'c')
            print(f"  Nombre de clients : {n_clients}")
            print(f"  Capacité véhicule : {cap}, Batterie Q : {Q}, Consommation h : {h}")

            best, cost = SA_algorithm(nodes, Q, h, cap)

            print(f"  Meilleure distance : {cost:.2f}")
            afficher_solution(best, nodes, Q, h)
            print("="*60, "\n")


# ============================================================
#  MAIN
# ============================================================

if __name__ == "__main__":
    dataset_folder = "/content/2E-EVRP-Instances-v2/manilakbay-2E-EVRP-Instances-dc2c2d0/Type_x/Customer_10"
    traiter_toutes_les_instances(dataset_folder)


Traitement de R203_C10x.txt
  Nombre de clients : 10
  Capacité véhicule : 250.0, Batterie Q : 60.63, Consommation h : 1.0
  Meilleure distance : inf
  Paramètres : Q=60.63, h=1.0
  Tournée 1: Dépôt -> C7 -> C9 -> C3 -> C5 -> C6 -> F4 -> C0 -> C8 -> C1 -> F3 -> F0 -> S0 -> F1 -> F2 -> C4 -> C2 -> Dépôt
  Nombre total de tournées : 1

Traitement de C125_C10x_n.txt
  Nombre de clients : 10
  Capacité véhicule : 100.0, Batterie Q : 79.7, Consommation h : 1.0
  Meilleure distance : inf
  Paramètres : Q=79.7, h=1.0
  Tournée 1: Dépôt -> C46 -> S6 -> C68 -> C74 -> C81 -> F12 -> C13 -> C61 -> C35 -> F16 -> Dépôt
  Tournée 2: Dépôt -> C71 -> F_sS6 -> F14 -> C8 -> F7 -> C53 -> Dépôt
  Nombre total de tournées : 2

Traitement de RC132_C10x_n.txt
  Nombre de clients : 10
  Capacité véhicule : 100.0, Batterie Q : 79.7, Consommation h : 1.0
  Meilleure distance : 801.74
  Paramètres : Q=79.7, h=1.0
  Tournée 1: Dépôt -> S6 -> F18 -> C38 -> C3 -> F11 -> C36 -> C53 -> F6 -> C68 -> C16 -> F2 -> C15 ->